<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/efficientnet_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**test gpu**

In [ ]:
!nvidia-smi

**imports**

In [23]:
import warnings
warnings.filterwarnings("ignore")

# keras and tf
import tensorflow as tf
import keras

# models
from keras.models import Sequential
from keras.models import Model

# backend
from keras import optimizers, metrics, models
import keras.backend as K

# layers
from keras.layers import Input, add, Conv2D, Flatten, Dense, Dropout, Activation
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

# optimizers
from keras.optimizers import SGD, Adam

# regularizers
from keras.regularizers import l2

# training
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler

# save
import h5py

# keras aplications
from keras.applications import DenseNet201, DenseNet169, InceptionResNetV2, ResNet152V2, InceptionV3, DenseNet121, Xception, MobileNet, VGG19


# other libs
import math
import pickle
import os

**install efficieintNet**

In [ ]:
pip install -U efficientnet

**import efficientNet**

In [4]:
import efficientnet.keras as efn 

**connect to the drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**set options**

In [28]:
image_size = 224 
batch_size = 8
nr_classes = 50
nr_ephocs = 30
train_dir = "/content/drive/My Drive/Datasets/food_data_50/50/train"
test_dir = "/content/drive/My Drive/Datasets/food_data_50/50/test"
all_dir = "/content/drive/My Drive/Datasets/food_data_50/50/all"
model_path = "/content/drive/My Drive/models/efficientnetb7_keras_50_class_model.hdf5"

**generate data**

In [ ]:
train_datagen = ImageDataGenerator(
        # rotation -40 40
        rotation_range=40,

        # shift images and fill pixels with nearest
        width_shift_range=0.2,
        height_shift_range=0.2,
        fill_mode='nearest',

        horizontal_flip=True,
        zoom_range=0.3,

        shear_range=0.2,

        brightness_range=[0.1, 0.5],
        channel_shift_range=0.5,

        # normalization 
        featurewise_center=True,
        featurewise_std_normalization=True,
        # validation_split=0.2,
        )

valid_datagen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)

train = train_datagen.flow_from_directory(train_dir, target_size=(image_size,image_size), batch_size=batch_size)
valid = valid_datagen.flow_from_directory(test_dir, target_size=(image_size,image_size), batch_size=batch_size)

# train = train_datagen.flow_from_directory(
#     directory = all_dir,
#     target_size = (image_size,image_size), 
#     batch_size = batch_size,
#     subset="training",
#     shuffle=True
# )

# valid = train_datagen.flow_from_directory(
#     directory = all_dir,
#     target_size = (image_size,image_size), 
#     batch_size = batch_size,
#     subset="validation",
#     shuffle=True
# )

**build model**

In [ ]:
base_model = efn.EfficientNetB0(include_top=False)

In [ ]:
base_model = efn.EfficientNetB7(include_top=False)

In [20]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
preds=Dense(nr_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=preds)

**set optimizer**

In [ ]:
opt = SGD(lr=0.0001, momentum=0.9)

In [ ]:
opt = SGD(lr=0.01, nesterov=True)

In [ ]:
opt = Adam(lr=0.0001)

In [ ]:
opt = Adagrad()

In [ ]:
opt = RMSprop()

**compile model**

In [25]:
import functools
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=5)
top5_acc.__name__ = 'top5_acc'

top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=10)
top10_acc.__name__ = 'top10_acc'

model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy', top3_acc, top5_acc, top10_acc])

In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])

**set checkpointer**

In [26]:
checkpointer = ModelCheckpoint(filepath= model_path, monitor='val_loss', save_best_only=True)

**train**

In [ ]:
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid, callbacks=[checkpointer])

In [ ]:
history = model.fit_generator(train, epochs=nr_ephocs, validation_data=valid)